### <p style="font-family: Arial; color: gold; font-weight: bold;">**create by Tom Tan in 8.30.2024** </p>
##### Now one notebook will deal with all the prefix, the downside is that the define properties will be the same for all the prefix.

***
# **1. Imports**

In [1]:
import os, re, glob
import pandas as pd

import get_properties_functions_for_WI as gp

common_structure_folder = "1.common_structure"
log_files_folder = "2.log_files"
sdf_files_folder = "3.sdf_files"
temp_folder = "temp"
atom_mappings_folder = "4.atom_mappings"
output_folder = "5.postprocessed_results"

***
# **2. Import the atom map from preprocess notebook**
### <p style="font-family: Arial; color: gold; font-weight: bold;"> **Grep all available prefix based on the log file** </p>

In [2]:
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

In [3]:
atom_map_df_all = {}
for prefix in prefixs:
    atom_mappings = atom_mappings_folder + os.sep + prefix + "_atom_map.xlsx"
    atom_map_df = pd.read_excel(
        atom_mappings, "Sheet1", index_col=0, header=0, engine="openpyxl"
    )
    # add the log_files_folder before all the log_names cells
    atom_map_df["log_name"] = log_files_folder + os.sep + atom_map_df["log_name"]
    atom_map_df_all[prefix] = atom_map_df.copy(deep=True)
    print(f"prefix: {prefix}")
    display(atom_map_df.head(3))

prefix: pyrdz


,log_name,N3,N4,C5,C6,C7,C2,C1,H1
0,2.log_files\pyrdz1_conf-1_anion,N7,N6,C5,C4,C3,C2,C1,H8
1,2.log_files\pyrdz2_conf-1_anion,C3,N4,N5,C6,C7,C2,C1,H8
2,2.log_files\pyrdz3_conf-1_anion,N8,N7,C6,C5,C4,C3,C2,H12


prefix: pyrd


,log_name,C3,C4,N5,C6,C7,C2,C1,H1
0,2.log_files\pyrd1_conf-1_anion,C5,C4,N3,C11,C10,C2,C1,H12
1,2.log_files\pyrd2_conf-1_anion,C11,N10,C9,C4,C3,C2,C1,H12
2,2.log_files\pyrd3_conf-1_anion,C3,C4,N5,C6,C7,C2,C1,H12


prefix: pyrmd


,log_name,N3,C4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrmd1_conf-1_anion,N3,C4,C5,C10,N11,C2,C1,H12
1,2.log_files\pyrmd2_conf-1_anion,C10,N9,C8,C3,N11,C2,C1,H12
2,2.log_files\pyrmd3_conf-2_anion,N4,C5,C10,C11,N12,C3,C2,H16


prefix: pyrz


,log_name,C3,N4,C5,C6,N7,C2,C1,H1
0,2.log_files\pyrz1_conf-1_anion,C11,N10,C5,C4,N3,C2,C1,H12
1,2.log_files\pyrz2_conf-1_anion,C12,N11,C10,C5,N4,C3,C2,H16
2,2.log_files\pyrz3_conf-1_anion,C7,N6,C5,C4,N3,C2,C1,H8


# **3. Define Properties to Collect**
### <p style="font-family: Arial; color: gold"> !!!User input required, Change/comment the properties block to the one you want to collect. </p>

In [4]:
for prefix, _ in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    pd.set_option("display.max_columns", None)
    # ---------------GoodVibes Engergies---------------
    # uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
    # calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
    # inputs: dataframe, temperature
    df = atom_map_df_all[prefix].copy(deep=True)
    df = gp.get_goodvibes_e(df, 298.15)

    # ---------------Frontier Orbitals-----------------
    # E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
    df = gp.get_frontierorbs(df)

    # ---------------Polarizability--------------------
    # Exact polarizability
    df = gp.get_polarizability(df)

    # ---------------Dipole----------------------------
    # Total dipole moment magnitude in Debye
    df = gp.get_dipole(df)

    # ---------------Volume----------------------------
    # Molar volume
    # requires the Gaussian keyword = "volume" in the .com file
    df = gp.get_volume(df)

    # ---------------SASA------------------------------
    # Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
    df = gp.get_SASA(df)

    # ---------------NBO-------------------------------
    # natural charge from NBO
    # requires the Gaussian keyword = "pop=nbo7" in the .com file
    nbo_list = ["C1", "C2"]
    df = gp.get_nbo(df, nbo_list)

    # ---------------NMR-------------------------------
    # isotropic NMR shift
    # requires the Gaussian keyword = "nmr=giao" in the .com file
    # nmr_list = ["C1", "C2"]
    # df = gp.get_nmr(df, nmr_list)

    # ---------------Distance--------------------------
    # distance between 2 atoms
    dist_list_of_lists = [["C1", "C2"]]
    df = gp.get_distance(df, dist_list_of_lists)

    # ---------------Angle-----------------------------
    # angle between 3 atoms
    # angle_list_of_lists = [["C5", "N1", "C1"]]
    # df = gp.get_angles(df, angle_list_of_lists)

    # ---------------Dihedral--------------------------
    # dihedral angle between 4 atoms
    # dihedral_list_of_lists = [["C4", "C5", "N1", "C1"], ["C2", "C1", "N1", "C5"]]
    # df = gp.get_dihedral(df, dihedral_list_of_lists)

    # ---------------Vbur Scan-------------------------
    # uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
    # inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
    # if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
    vbur_list = ["C1", "C2"]
    df = gp.get_vbur_scan(df, vbur_list, 2, 2, 0.5)

    # ---------------Sterimol morfeus------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values
    # NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
    sterimol_list_of_lists = [["C1", "C2"]]
    df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists)

    # ---------------Buried Sterimol-------------------
    # uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
    # atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
    # for more information: https://kjelljorner.github.io/morfeus/sterimol.html
    # inputs: dataframe, list of atom pairs, r_buried
    # sterimol_list_of_lists = [["C1", "C2"]]
    # df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5)

    # ---------------Sterimol DBSTEP-------------------
    # uses DBSTEP to calculate Sterimol L, B1, and B5 values
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
    # sterimol_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists)

    # ---------------Sterimol2Vec----------------------
    # uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size
    # default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
    # more info here: https://github.com/patonlab/DBSTEP
    # inputs: dataframe, list of atom pairs, end_radius, and step_size
    # sterimol2vec_list_of_lists = [["N1", "C1"], ["N1", "C5"]]
    # df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0)

    # ---------------Pyramidalization------------------
    # uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
    # collects values based on two definitions of pyramidalization
    # details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
    # pyr_list = ["C1"]
    # df = gp.get_pyramidalization(df, pyr_list)

    # ---------------Plane Angle-----------------------
    # !plane angle between 2 planes (each defined by 6 atoms)
    # planeangle_list_of_lists = [["N1", "C1", "C5"], ["C2", "C3", "C4"]]
    # df = gp.get_planeangle(df, planeangle_list_of_lists)

    # --------------LP energy - custom from first cell---------------
    # lp_list = ["N1"]
    # df = gp.get_one_lp_energy(df, lp_list)

    # ---------------Time----------------------------------
    # returns the total CPU time and total Wall time (not per subjob) because we are pioneers
    # if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
    # df = gp.get_time(df)

    # ---------------ChelpG----------------------------
    # ChelpG ESP charge
    # requires the Gaussian keyword = "pop=chelpg" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_chelpg(df, a_list)

    # ---------------Hirshfeld-------------------------
    # Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
    # requires the Gaussian keyword = "pop=hirshfeld" in the .com file
    # a_list = ["C1", "C2", "C3", "C4", "C5", "N1"]
    # df = gp.get_hirshfeld(df, a_list)

    # !new functions below!
    # ---------------Natural Bond Order (total/covalent/ionic)-------------------------
    # Natural Bond Order (total/covalent/ionic) between 2 atoms, might return non-numerical values
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_bond_order_list = [["C1", "C2"]]
    df = gp.get_natural_bond_order(df, natural_bond_order_list)

    # ---------------Natural Atomic Valencies, Electron Counts, and Charges-------------------------
    # Natural Atomic Valencies, Electron Counts, and Charges of a atom
    # requires Natural Resonance Theory Analysis in Gaussian input file ("$nbo nrt $end" in the .com file)
    natural_atomic_valencies_list = ["C1", "C2"]
    df = gp.get_natural_atomic_valencies(df, natural_atomic_valencies_list)

    display(df)
    # copy the changes back to atom_map_df_all
    atom_map_df_all[prefix] = df.copy(deep=True)
    
# delete the "Goodvibes_output.dat" temp file
if os.path.exists("Goodvibes_output.dat"):
    os.remove("Goodvibes_output.dat")

processing prefix: pyrdz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrdz1_conf-1_anion,N7,N6,C5,C4,C3,C2,C1,H8,-302.982492,0.088762,-302.887089,0.035899,0.035845,-302.922988,-302.922934,298.15,-0.02285,0.17703,0.077090,0.19988,0.01487,87.3665,74.3950,4.3600,973.223,247.241329,331.668176,0.937200,-0.68988,0.09093,1.37265,85.098140,95.796746,6.702696,1.700000,3.280112,1.5108,1.2951,0.2157,3.0584,0.6203,-0.2573,3.3362,0.4938,-0.1559
1,2.log_files\pyrdz2_conf-1_anion,C3,N4,N5,C6,C7,C2,C1,H8,-302.995475,0.089531,-302.899448,0.035548,0.035546,-302.934996,-302.934994,298.15,-0.03489,0.19082,0.077965,0.22571,0.01347,85.7330,70.1348,2.1685,896.162,246.201207,330.946590,0.939794,-0.66830,-0.07411,1.37234,85.211131,96.816890,6.216797,1.700527,3.275876,1.5335,1.3160,0.2175,3.0303,0.5608,-0.3223,3.5204,0.3562,-0.0593
2,2.log_files\pyrdz3_conf-1_anion,N8,N7,C6,C5,C4,C3,C2,H12,-342.289476,0.116657,-342.164510,0.040186,0.039862,-342.204695,-342.204372,298.15,-0.01941,0.17521,0.077900,0.19462,0.01559,101.8870,87.9460,3.1358,1106.008,277.171300,380.768558,0.916593,-0.43009,0.09398,1.37634,92.171358,95.732180,6.710076,1.832918,3.283179,1.4930,1.3277,0.1653,3.3513,0.4731,-0.1757,3.3759,0.4330,-0.1479


processing prefix: pyrd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrd1_conf-1_anion,C5,C4,N3,C11,C10,C2,C1,H12,-440.620459,0.148369,-440.463017,0.041819,0.041685,-440.504836,-440.504702,298.15,-0.03713,0.14427,0.053570,0.18140,0.00791,151.5260,159.9430,4.5298,1231.496,321.262839,464.407217,0.902727,-0.63176,0.12919,1.37220,85.094912,95.774148,8.897560,1.700371,4.404027,1.5642,1.3606,0.2036,2.9743,0.5491,-0.2896,3.4047,0.5367,-0.0333
1,2.log_files\pyrd2_conf-1_anion,C11,N10,C9,C4,C3,C2,C1,H12,-440.613490,0.148143,-440.456235,0.041841,0.041724,-440.498076,-440.497959,298.15,-0.02826,0.13999,0.055865,0.16825,0.00927,157.3340,177.7960,3.2742,1416.892,320.432780,464.008075,0.904546,-0.65663,-0.08946,1.37697,85.224044,96.800749,9.043758,1.700775,4.214565,1.5454,1.3262,0.2193,3.0693,0.6075,-0.2527,3.5178,0.3552,-0.0673
2,2.log_files\pyrd3_conf-1_anion,C3,C4,N5,C6,C7,C2,C1,H12,-440.625214,0.148757,-440.467396,0.042017,0.041694,-440.509414,-440.509091,298.15,-0.04140,0.14908,0.053840,0.19048,0.00761,140.0700,107.2940,2.6083,1386.894,315.022765,458.374623,0.912618,-0.61606,-0.01610,1.36940,85.482309,96.787836,6.998738,1.701001,5.742282,1.5937,1.3852,0.2085,3.1061,0.5667,-0.2805,3.5553,0.3499,-0.0444
3,2.log_files\pyrd4_conf-1_anion,C5,C4,N3,C11,C6,C2,C1,H12,-440.551757,0.145472,-440.397443,0.042001,0.041451,-440.439444,-440.438894,298.15,-0.01657,0.11603,0.049730,0.13260,0.00933,135.1830,93.6876,10.9549,1372.069,316.442835,461.071851,0.912083,-1.00914,0.31416,1.48572,78.706095,95.461002,7.093010,1.870356,5.729801,1.1107,0.8840,0.2267,2.5432,0.4971,-0.8981,3.1776,0.7411,-0.0549
4,2.log_files\pyrd5_conf-1_anion,C3,N11,C10,C9,C4,C2,C1,H12,-440.611323,0.148148,-440.454042,0.041970,0.041811,-440.496012,-440.495853,298.15,-0.02771,0.13648,0.054385,0.16419,0.00901,155.4940,169.4700,4.9777,1096.479,320.602786,464.296392,0.904441,-0.63992,0.12582,1.37118,85.165935,95.819344,9.015736,1.700070,4.356016,1.5764,1.3737,0.2027,2.9654,0.5467,-0.3400,3.4042,0.5304,-0.0654
5,2.log_files\pyrd6_conf-1_anion,C3,N4,C5,C6,C11,C2,C1,H12,-440.559375,0.146294,-440.404357,0.041353,0.041235,-440.445710,-440.445593,298.15,-0.02085,0.11107,0.045110,0.13192,0.00771,136.0480,96.6492,10.1046,1456.682,315.242707,460.163707,0.914353,-1.01903,0.02686,1.49433,79.322701,96.581224,7.204977,1.872913,5.682147,1.1286,0.8960,0.2327,2.5447,0.5064,-0.9071,3.5774,0.3954,-0.0046
6,2.log_files\pyrd7_conf-1_anion,C10,C5,N4,C12,C11,C3,C2,H16,-479.929078,0.176233,-479.742060,0.046020,0.045569,-479.788079,-479.787629,298.15,-0.03386,0.14462,0.055380,0.17848,0.00859,167.2320,172.5580,2.7262,1484.938,350.872814,513.360452,0.883655,-0.37646,0.12913,1.37570,92.242381,95.767691,8.906018,1.837292,4.384873,1.5291,1.3787,0.1504,3.3886,0.4567,-0.1547,3.4101,0.4699,-0.0830
7,2.log_files\pyrd8_conf-1_anion,C12,N11,C10,C5,C4,C3,C2,H16,-479.922161,0.175894,-479.735423,0.046124,0.045664,-479.781546,-479.781087,298.15,-0.02678,0.14047,0.056845,0.16725,0.00966,173.4720,189.7880,1.7727,1327.700,348.462598,512.263951,0.888499,-0.40168,-0.09125,1.37935,92.145532,96.787836,9.037872,1.809395,4.228558,1.5115,1.3522,0.1593,3.3616,0.4547,-0.1825,3.5615,0.3116,-0.0550
8,2.log_files\pyrd9_conf-2_anion,C4,C5,N6,C7,C12,C3,C2,H16,-479.932617,0.176526,-479.745262,0.046304,0.045689,-479.791567,-479.790951,298.15,-0.03972,0.14832,0.054300,0.18804,0.00784,156.4900,124.0840,2.4114,1517.823,342.642599,505.771780,0.895940,-0.37350,-0.01892,1.37210,92.303719,96.749096,6.992450,1.8

processing prefix: pyrmd
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrmd1_conf-1_anion,N3,C4,C5,C10,N11,C2,C1,H12,-456.671952,0.136929,-456.526108,0.041726,0.041480,-456.567834,-456.567588,298.15,-0.04437,0.13188,0.043755,0.17625,0.00543,144.9480,152.6310,5.4324,1138.958,315.992408,453.847616,0.903818,-0.61311,0.27507,1.36554,85.049716,94.647469,8.874571,1.700000,4.353035,1.5684,1.3983,0.1702,3.0333,0.5547,-0.2657,3.1798,0.7120,-0.0943
1,2.log_files\pyrmd2_conf-1_anion,C10,N9,C8,C3,N11,C2,C1,H12,-456.611366,0.134259,-456.468473,0.041500,0.041115,-456.509973,-456.509588,298.15,-0.02887,0.10691,0.039020,0.13578,0.00561,128.2540,89.0171,9.5937,1357.463,311.012388,450.272903,0.913462,-1.01203,0.35102,1.47855,78.425232,95.532025,7.131071,1.864369,5.718808,1.1405,0.8987,0.2418,2.5331,0.5196,-0.8775,3.1272,0.7990,-0.0738
2,2.log_files\pyrmd3_conf-2_anion,N4,C5,C10,C11,N12,C3,C2,H16,-495.980623,0.164782,-495.805216,0.045903,0.045326,-495.851120,-495.850542,298.15,-0.04070,0.13212,0.045710,0.17282,0.00605,160.9100,167.0760,3.5445,1550.788,345.662381,502.673608,0.884483,-0.36518,0.27582,1.36911,92.151989,94.573218,8.884547,1.837927,4.339318,1.5570,1.4298,0.1272,3.3457,0.4118,-0.1728,3.2179,0.6697,-0.1099
3,2.log_files\pyrmd4_conf-2_anion,N10,C11,C9,C4,N12,C3,C2,H16,-495.994032,0.165441,-495.817937,0.046691,0.045662,-495.864628,-495.863599,298.15,-0.05370,0.14450,0.045400,0.19820,0.00520,147.7440,117.3270,3.6540,1345.361,339.042316,495.895564,0.893629,-0.34115,0.14763,1.36674,92.358600,95.770919,6.984591,1.904732,5.761006,1.6343,1.5093,0.1250,3.3963,0.3544,-0.2145,3.4804,0.4325,-0.0477
4,2.log_files\pyrmd5_conf-1_anion,N3,C4,C5,C6,N7,C2,C1,H8,-303.024097,0.089836,-302.927734,0.035790,0.035678,-302.963524,-302.963412,298.15,-0.02782,0.18907,0.080625,0.21689,0.01499,85.7603,71.8606,3.4934,663.943,248.461518,332.134423,0.933471,-0.66244,0.27959,1.36766,84.998063,94.663611,6.721477,1.700000,3.251091,1.5411,1.3571,0.1840,3.1130,0.5505,-0.2980,3.1272,0.7234,-0.1494
5,2.log_files\pyrmd6_conf-1_anion,C4,N3,C7,C6,N5,C2,C1,H8,-303.034139,0.090544,-302.937192,0.035424,0.035414,-302.972616,-302.972606,298.15,-0.03763,0.19743,0.079900,0.23506,0.01358,83.4261,66.3166,1.9232,850.901,247.151405,331.195799,0.936651,-0.65954,0.13705,1.36818,85.191761,95.838714,6.178909,1.700000,3.284947,1.5438,1.3429,0.2009,3.0939,0.5510,-0.3551,3.3458,0.5268,-0.0910
6,2.log_files\pyrmd7_conf-1_anion,C5,N4,C3,C7,N6,C2,C1,H8,-303.016355,0.089465,-302.920301,0.035731,0.035720,-302.956032,-302.956021,298.15,-0.01684,0.18207,0.082615,0.19891,0.01716,88.8721,76.1439,0.3404,950.160,246.061410,330.576662,0.939627,-0.71770,-0.12120,1.37828,85.253099,96.781379,6.741480,1.700212,3.267593,1.5285,1.2909,0.2377,2.9785,0.6813,-0.2543,3.5196,0.3272,-0.1485
7,2.log_files\pyrmd8_conf-1_anion,N4,C5,C6,C7,N8,C3,C2,H12,-342.331591,0.117706,-342.205710,0.039894,0.039584,-342.245605,-342.245294,298.15,-0.02413,0.18707,0.081470,0.21120,0.01571,100.3780,84.9092,1.7129,864.215,278.281492,381.142725,0.913534,-0.41136,0.28205,1.37107,92.181043,94.563533,6.724709,1.831835,3.286608,1.5279,1.3893,0.1386,3.3754,0.4191,-0.2055,3.1660,0.6763,-0.1576
8,2.log_files\pyrmd9_conf-1_anion,C5,N4,C8,C7,N6,C3,C2,H12,-342.341797,0.118495,-342.215276,0.039435,0.039319,-342.254712,-342.254596,298.15,-0.03362,0.19568,0.081030,0.22930,0.01432,97.6797,77.9813,1.7701,986.312,277.031383,380.360394,0.916401,-0.40567,0.13825,1.37157,92.242381,95.761235,6.182972,1.831808,3.287497,1.

processing prefix: pyrz
Goodvibes function has completed
Frontier orbitals function has completed
Polarizability function has completed
Dipole function has completed
Volume function has completed
SASA function has completed
NBO function has completed for ['C1', 'C2']
Distance function has completed for [['C1', 'C2']]
Vbur scan function has completed for ['C1', 'C2'] from 2 to 2
Morfeus Sterimol function has completed for [['C1', 'C2']]
Natural Bond Order function has completed for [['C1', 'C2']]
Natural Atomic Valencies function has completed for ['C1', 'C2']


,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrz1_conf-1_anion,C11,N10,C5,C4,N3,C2,C1,H12,-456.671574,0.136787,-456.525880,0.041605,0.041422,-456.567486,-456.567303,298.15,-0.04597,0.13534,0.044685,0.18131,0.00551,146.2850,159.8690,3.2453,1458.058,315.872386,453.680224,0.903939,-0.62054,0.08555,1.37108,85.107825,95.832257,8.905194,1.701024,4.229301,1.5697,1.3704,0.1993,2.9987,0.5534,-0.2614,3.3937,0.5073,-0.0755
1,2.log_files\pyrz2_conf-1_anion,C12,N11,C10,C5,N4,C3,C2,H16,-495.980539,0.164668,-495.805226,0.045942,0.045369,-495.851168,-495.850595,298.15,-0.04244,0.13617,0.046865,0.17861,0.00615,161.8270,172.5720,1.2186,1620.160,345.572357,502.633882,0.884667,-0.36404,0.08456,1.37416,92.235925,95.806431,8.908226,1.835675,4.229679,1.5444,1.3950,0.1493,3.4114,0.4374,-0.1284,3.4107,0.4710,-0.0980
2,2.log_files\pyrz3_conf-1_anion,C7,N6,C5,C4,N3,C2,C1,H8,-303.016727,0.089581,-302.920611,0.035689,0.035654,-302.956301,-302.956265,298.15,-0.02379,0.18320,0.079705,0.20699,0.01535,87.6117,75.7281,1.6637,1115.615,247.141445,331.282867,0.936852,-0.68138,0.09115,1.37277,85.230501,95.706353,6.721764,1.700000,3.287063,1.5370,1.3222,0.2149,3.0198,0.5929,-0.2900,3.3372,0.5073,-0.1320
3,2.log_files\pyrz4_conf-1_anion,C8,N7,C6,C5,N4,C3,C2,H12,-342.322237,0.117395,-342.196621,0.040092,0.039661,-342.236713,-342.236282,298.15,-0.02096,0.17780,0.078420,0.19876,0.01547,103.2530,89.8004,0.7106,1033.708,275.331271,379.423603,0.920545,-0.43602,0.09438,1.37529,92.106792,95.799974,6.725155,1.801828,3.287877,1.5396,1.3687,0.1709,3.1665,0.4438,-0.2880,3.4192,0.4771,-0.0798


## 3.1 Save collected properties to Excel and pickle file

In [5]:
for prefix, df in atom_map_df_all.items():
    # save the pandas dataframe to a xlsx file
    with pd.ExcelWriter(temp_folder + os.sep + prefix + "_extracted_properties.xlsx") as writer:
        df.to_excel(writer)

# **4. Post-processing**

In [6]:
import re
import pandas as pd
import numpy as np
from tabulate import tabulate

In [7]:
# for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
# this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
suffix = "_"
prefixs = {}
for file in glob.glob("*.xlsx", root_dir=atom_mappings_folder):
    key = re.search(r"^(\D+)_atom_map", file)
    if key and key.group(1) in prefixs:
        prefixs[key.group(1)].append(file)
    else:
        prefixs[key.group(1)] = [file]

# columns that provide atom mapping information are dropped, not need if these columns contain cells that cannot be convert to float
# e.g. atom_columns_to_drop = ["C3", "C4", "C5", "N1", "C1", "C2"]
atom_columns_to_drop = []

# title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

##### If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

In [8]:
atom_map_df_all = {}

for prefix in prefixs:
    df = pd.read_excel(
        temp_folder + os.sep +
        prefix + "_extracted_properties.xlsx",
        "Sheet1",
        index_col=0,
        header=0,
        engine="openpyxl",
    )
    display(df.head(2))
    atom_map_df_all[prefix] = df.copy(deep=True)

,log_name,N3,N4,C5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrdz1_conf-1_anion,N7,N6,C5,C4,C3,C2,C1,H8,-302.982492,0.088762,-302.887089,0.035899,0.035845,-302.922988,-302.922934,298.15,-0.02285,0.17703,0.077090,0.19988,0.01487,87.3665,74.3950,4.3600,973.223,247.241329,331.668176,0.937200,-0.68988,0.09093,1.37265,85.098140,95.796746,6.702696,1.700000,3.280112,1.5108,1.2951,0.2157,3.0584,0.6203,-0.2573,3.3362,0.4938,-0.1559
1,2.log_files\pyrdz2_conf-1_anion,C3,N4,N5,C6,C7,C2,C1,H8,-302.995475,0.089531,-302.899448,0.035548,0.035546,-302.934996,-302.934994,298.15,-0.03489,0.19082,0.077965,0.22571,0.01347,85.7330,70.1348,2.1685,896.162,246.201207,330.946590,0.939794,-0.66830,-0.07411,1.37234,85.211131,96.816890,6.216797,1.700527,3.275876,1.5335,1.3160,0.2175,3.0303,0.5608,-0.3223,3.5204,0.3562,-0.0593


,log_name,C3,C4,N5,C6,C7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrd1_conf-1_anion,C5,C4,N3,C11,C10,C2,C1,H12,-440.620459,0.148369,-440.463017,0.041819,0.041685,-440.504836,-440.504702,298.15,-0.03713,0.14427,0.053570,0.18140,0.00791,151.526,159.943,4.5298,1231.496,321.262839,464.407217,0.902727,-0.63176,0.12919,1.37220,85.094912,95.774148,8.897560,1.700371,4.404027,1.5642,1.3606,0.2036,2.9743,0.5491,-0.2896,3.4047,0.5367,-0.0333
1,2.log_files\pyrd2_conf-1_anion,C11,N10,C9,C4,C3,C2,C1,H12,-440.613490,0.148143,-440.456235,0.041841,0.041724,-440.498076,-440.497959,298.15,-0.02826,0.13999,0.055865,0.16825,0.00927,157.334,177.796,3.2742,1416.892,320.432780,464.008075,0.904546,-0.65663,-0.08946,1.37697,85.224044,96.800749,9.043758,1.700775,4.214565,1.5454,1.3262,0.2193,3.0693,0.6075,-0.2527,3.5178,0.3552,-0.0673


,log_name,N3,C4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrmd1_conf-1_anion,N3,C4,C5,C10,N11,C2,C1,H12,-456.671952,0.136929,-456.526108,0.041726,0.041480,-456.567834,-456.567588,298.15,-0.04437,0.13188,0.043755,0.17625,0.00543,144.948,152.6310,5.4324,1138.958,315.992408,453.847616,0.903818,-0.61311,0.27507,1.36554,85.049716,94.647469,8.874571,1.700000,4.353035,1.5684,1.3983,0.1702,3.0333,0.5547,-0.2657,3.1798,0.712,-0.0943
1,2.log_files\pyrmd2_conf-1_anion,C10,N9,C8,C3,N11,C2,C1,H12,-456.611366,0.134259,-456.468473,0.041500,0.041115,-456.509973,-456.509588,298.15,-0.02887,0.10691,0.039020,0.13578,0.00561,128.254,89.0171,9.5937,1357.463,311.012388,450.272903,0.913462,-1.01203,0.35102,1.47855,78.425232,95.532025,7.131071,1.864369,5.718808,1.1405,0.8987,0.2418,2.5331,0.5196,-0.8775,3.1272,0.799,-0.0738


,log_name,C3,N4,C5,C6,N7,C2,C1,H1,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,G(T)_spc(Hartree),qh_G(T)_spc(Hartree),T,HOMO,LUMO,μ,η,ω,polar_iso(Debye),polar_aniso(Debye),dipole(Debye),volume(Bohr_radius³/mol),SASA_surface_area(Å²),SASA_volume(Å³),SASA_sphericity,NBO_charge_C1,NBO_charge_C2,distance_C1_C2(Å),%Vbur_C1_2.0Å,%Vbur_C2_2.0Å,Sterimol_L_C1_C2(Å)_morfeus,Sterimol_B1_C1_C2(Å)_morfeus,Sterimol_B5_C1_C2(Å)_morfeus,C1_C2_bond_order_total,C1_C2_bond_order_covalent,C1_C2_bond_order_ionic,C1_Co-Valency,C1_Electro-Valency,C1_FormalCharge,C2_Co-Valency,C2_Electro-Valency,C2_FormalCharge
0,2.log_files\pyrz1_conf-1_anion,C11,N10,C5,C4,N3,C2,C1,H12,-456.671574,0.136787,-456.525880,0.041605,0.041422,-456.567486,-456.567303,298.15,-0.04597,0.13534,0.044685,0.18131,0.00551,146.285,159.869,3.2453,1458.058,315.872386,453.680224,0.903939,-0.62054,0.08555,1.37108,85.107825,95.832257,8.905194,1.701024,4.229301,1.5697,1.3704,0.1993,2.9987,0.5534,-0.2614,3.3937,0.5073,-0.0755
1,2.log_files\pyrz2_conf-1_anion,C12,N11,C10,C5,N4,C3,C2,H16,-495.980539,0.164668,-495.805226,0.045942,0.045369,-495.851168,-495.850595,298.15,-0.04244,0.13617,0.046865,0.17861,0.00615,161.827,172.572,1.2186,1620.160,345.572357,502.633882,0.884667,-0.36404,0.08456,1.37416,92.235925,95.806431,8.908226,1.835675,4.229679,1.5444,1.3950,0.1493,3.4114,0.4374,-0.1284,3.4107,0.4710,-0.0980


## 4.1 Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [9]:
compound_list_all = {}

for prefix, df in atom_map_df_all.items():
    print(f"processing prefix: {prefix}")
    compound_list = []

    for index, row in df.iterrows():
        log_file = row["log_name"]  # read file name from df
        # first split by "\" take the last part
        log_file = log_file.split(os.sep)[-1]
        prefix_and_compound = log_file.split(str(suffix))
        compound = prefix_and_compound[0].split(str(prefix))  # splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
        compound_list.append(compound[1])

    compound_list = list(set(compound_list))  # removes duplicate stuctures that result from having conformers of each
    compound_list.sort(key=lambda x: int(re.search(r"\d+", x).group()))  # reorders numerically (not sure if it reorders alphabetically)
    print(f"items numbering: {compound_list}")
    compound_list_all[prefix] = compound_list

    # this should generate a list that looks like this: ['24', '27', '34', '48']

processing prefix: pyrdz
items numbering: ['1', '2', '3']
processing prefix: pyrd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']
processing prefix: pyrmd
items numbering: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
processing prefix: pyrz
items numbering: ['1', '2', '3', '4']


## 4.2 Post-processing to get properties for each compound

##### changes made in 8/30/2024 <br> 1. avoid divide by zero error in the Boltzmann averaging, the original code had the if block order reversed, which caused the error. <br> 2. data cleaning by remove columns contain cell that cannot be converted to float. <br> 3. concat all data into row before concat them into the final dataframe. The originl modify individual cells which result in fragmented and raise performance warning.

In [10]:
all_df_master_for_all_prefixes = {}
properties_df_master_for_all_prefixes = {}

for prefix, df in atom_map_df_all.items():
    compound_list = compound_list_all[prefix]
    all_df_master = pd.DataFrame(columns=[])
    properties_df_master = pd.DataFrame(columns=[])

    for compound in compound_list:
        # defines the common start to all files using the input above
        substring = log_files_folder + os.sep + str(prefix) + str(compound) + str(suffix)

        # makes a data frame for one compound at a time for post-processing
        valuesdf = df[df["log_name"].str.startswith(substring)]
        valuesdf = valuesdf.drop(columns=atom_columns_to_drop)
        valuesdf = valuesdf.reset_index(
            drop=True
        )  # you must re-index otherwise the 2nd, 3rd, etc. compounds fail

        # filter column that are characters, we will attempt to convert them to numeric numbers, if fail, we will drop them
        for column in valuesdf:
            try:
                # exclude column "log_name"
                if column == "log_name":
                    continue
                valuesdf[column] = pd.to_numeric(valuesdf[column])
            except:
                print(f"Column {column} contains non-numeric values")
                valuesdf = valuesdf.drop(columns=column)
                valuesdf = valuesdf.reset_index(
                    drop=True
                )  # reset the index after dropping columns

        # define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
        non_boltz_columns = [
            "G(Hartree)",
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]  # don't boltzman average columns containing these strings in the column label
        reg_avg_columns = [
            "CPU_time_total(hours)",
            "Wall_time_total(hours)",
        ]  # don't boltzmann average these either, we average them in case that is helpful
        gv_extra_columns = [
            "G(T)_spc(Hartree)",
        ]
        gv_extra_columns.remove(str(energy_col_header))

        # calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
        valuesdf["∆G(Hartree)"] = (
            valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
        )
        valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
        valuesdf["e^(-∆G/RT)"] = np.exp(
            (valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)
        )  # R is in cal/(K*mol)
        valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
        values_boltz_row = []
        values_min_row = []
        values_max_row = []
        values_boltz_stdev_row = []
        values_range_row = []
        values_exclude_columns = []

        for column in valuesdf:
            if "log_name" in column:
                values_boltz_row.append("Boltzmann Averages")
                values_min_row.append("Ensemble Minimum")
                values_max_row.append("Ensemble Maximum")
                values_boltz_stdev_row.append("Boltzmann Standard Deviation")
                values_range_row.append("Ensemble Range")
                values_exclude_columns.append(column)  # used later to build final dataframe
            elif any(phrase in column for phrase in non_boltz_columns) or any(
                phrase in column for phrase in gv_extra_columns
            ):
                values_boltz_row.append("")
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            elif any(phrase in column for phrase in reg_avg_columns):
                values_boltz_row.append(
                    valuesdf[column].mean()
                )  # intended to print the average CPU/wall time in the boltz column
                values_min_row.append("")
                values_max_row.append("")
                values_boltz_stdev_row.append("")
                values_range_row.append("")
            else:
                valuesdf[column] = pd.to_numeric(
                    valuesdf[column]
                )  # to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
                values_boltz_row.append(
                    (valuesdf[column] * valuesdf["Mole Fraction"]).sum()
                )
                values_min_row.append(valuesdf[column].min())
                values_max_row.append(valuesdf[column].max())
                values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

                # this section generates the weighted std deviation (weighted by mole fraction)
                # formula: https://www.statology.org/weighted-standard-deviation-excel/

                boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum()  # number
                delta_values_sq = []

                # makes a list of the "deviation" for each conformer
                for index, row in valuesdf.iterrows():
                    value = row[column]
                    delta_value_sq = (value - boltz) ** 2
                    delta_values_sq.append(delta_value_sq)

                # w is list of weights (i.e. mole fractions)
                w = list(valuesdf["Mole Fraction"])
                # !swap the order here to avoid division by zero error
                if (
                    len(w) == 1
                ):  # if there is only one conformer in the ensemble, set the weighted standard deviation to 0
                    wstdev = 0
                # np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
                else:
                    wstdev = np.sqrt(
                        (np.average(delta_values_sq, weights=w))
                        / (((len(w) - 1) / len(w)) * np.sum(w))
                    )
                values_boltz_stdev_row.append(wstdev)

        valuesdf.loc[len(valuesdf)] = values_boltz_row
        valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
        valuesdf.loc[len(valuesdf)] = values_min_row
        valuesdf.loc[len(valuesdf)] = values_max_row
        valuesdf.loc[len(valuesdf)] = values_range_row

        # final output format is built here:
        explicit_order_front_columns = [
            "log_name",
            energy_col_header,
            "∆G(Hartree)",
            "∆G(kcal/mol)",
            "e^(-∆G/RT)",
            "Mole Fraction",
        ]

        # reorders the dataframe using front columns defined above
        valuesdf = valuesdf[
            explicit_order_front_columns
            + [
                col
                for col in valuesdf.columns
                if col not in explicit_order_front_columns
                and col not in values_exclude_columns
            ]
        ]

        # determine the index of the lowest energy conformer
        low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
        # copy the row to a new_row with the name of the log changed to Lowest E Conformer
        new_row = pd.DataFrame(valuesdf.loc[low_e_index[0]]).T
        new_row["log_name"] = "Lowest E Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # ------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------
        # if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
        # this template can be adjusted for min/max/etc.

        # find the index for the min or max column:
        ensemble_min_index = valuesdf[
            valuesdf["log_name"] == "Ensemble Minimum"
        ].index.tolist()

        # find the min or max value of the property (based on index above)
        # saves the value in a list (min_value) with one entry (this is why we call min_value[0])
        min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C1_2.0Å"].tolist()
        vbur_min_index = valuesdf[valuesdf["%Vbur_C1_2.0Å"] == min_value[0]].index.tolist()

        # copy the row to a new_row with the name of the log changed to Property_min_conformer
        new_row = pd.DataFrame(valuesdf.loc[vbur_min_index[0]]).T
        new_row["log_name"] = "%Vbur_C1_2.0Å_min_Conformer"

        valuesdf = pd.concat([valuesdf, new_row], ignore_index=True, axis=0)

        # --------------------------------------------------------------------------------------------------------------------

        # !here we define a list of properties we only want the minimal value for
        min_property_list = [
            "E_spc (Hartree)",
            "H_spc(Hartree)",
            "T",
            "T*S",
            "T*qh_S",
            "ZPE(Hartree)",
            "qh_G(T)_spc(Hartree)",
            "G(T)_spc(Hartree)",
        ]
        # extract the "Lowest E Conformer" row out of the dataframe
        Low_E_Conformer_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Lowest E Conformer"]
        )
        # extract the "Boltzmann Averages" row out of the dataframe
        Boltz_Avg_row = pd.DataFrame(
            valuesdf.loc[valuesdf["log_name"] == "Boltzmann Averages"]
        )
        # display(valuesdf) # debug display for finding the row index
        # display(Low_E_Conformer_row)

        # appends the frame to the master output
        all_df_master = pd.concat([all_df_master, valuesdf])

        # drop all the individual conformers
        dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
        valuesdf = valuesdf.drop(dropindex)
        valuesdf = valuesdf.reset_index(drop=True)

        # drop the columns created to determine the mole fraction and some that
        valuesdf = valuesdf.drop(columns=explicit_order_front_columns)
        try:
            valuesdf = valuesdf.drop(columns=gv_extra_columns)
        except:
            pass
        try:
            valuesdf = valuesdf.drop(columns=reg_avg_columns)
        except:
            pass

        # ---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------
        compound_name = prefix + str(compound)
        # --------------------------------------------------------------------------------------------------------------------

        properties_df = pd.DataFrame({"Compound_Name": [compound_name]})

        # builds a dataframe (for each compound) by adding summary properties as new columns
        for column in valuesdf:
            # print(column)
            # the indexes need to match the values dataframe - display it to double check if you need to make changes
            # (uncomment the display(valuesdf) in row 124 of this cell)

            # create a list of headers for the properties_df
            # if you're collecting properties for a specific conformer, edit the header to reflect that, it should match the order in the valuesdf log_name column
            if column in min_property_list:
                # ! if we are working with a property that we only want the minimum value for, we only need one header
                headers = [
                    f"{column}",
                ]
                # use data from the Low_E_Conformer_row
                row_dataframe = pd.DataFrame(
                    [Low_E_Conformer_row[column].values], columns=headers
                )
            else:
                headers = [
                    f"{column}_Boltz",
                ]
                row_dataframe = pd.DataFrame([Boltz_Avg_row[column].values], columns=headers)
            # Extract values for the current column from valuesdf and create a DataFrame
            # Display the DataFrame for verification
            # display(row_dataframe)
            # Concatenate the new DataFrame to the properties_df along the columns (axis=1)
            properties_df = pd.concat([properties_df, row_dataframe], axis=1)

        # concatenates the individual acid properties df into the master properties df
        properties_df_master = pd.concat([properties_df_master, properties_df], axis=0)

    # Reset the index of the master DataFrames
    all_df_master = all_df_master.reset_index(drop=True)
    all_df_master_for_all_prefixes[prefix] = all_df_master.copy(deep=True)
    properties_df_master = properties_df_master.reset_index(drop=True)
    properties_df_master_for_all_prefixes[prefix] = properties_df_master.copy(deep=True)
    
    # Print in tabulated format
    print(tabulate(properties_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))
    print(tabulate(all_df_master_for_all_prefixes[prefix], headers="keys", tablefmt="pretty"))

Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
Column N3 contains non-numeric values
Column N4 contains non-numeric values
Column C5 contains non-numeric values
Column C6 contains non-numeric values
Column C7 contains non-numeric values
Column C2 contains non-numeric values
Column C1 contains non-numeric values
Column H1 contains non-numeric values
+---+---------------+-----------------+--------------+----------------+----------+------

In [11]:
# merge all the properties_df_master_for_all_prefixes into a single dataframe, combine column with the same name
properties_df_master_for_all_prefixes_merged = pd.DataFrame(columns=[])
for prefix, df in properties_df_master_for_all_prefixes.items():
    display(df.head(1))
    properties_df_master_for_all_prefixes_merged = pd.concat([properties_df_master_for_all_prefixes_merged, df], axis=0)
    properties_df_master_for_all_prefixes_merged.reset_index(drop=True, inplace=True)

print(f"Combine summary properties for all prefixes: ")
display(properties_df_master_for_all_prefixes_merged.head(5))

,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-302.982492,0.088762,-302.887089,0.035899,0.035845,-302.922934,298.15,-0.02285,0.17703,0.07709,0.19988,0.01487,87.3665,74.395,4.36,973.223,247.241329,331.668176,0.9372,-0.68988,0.09093,1.37265,85.09814,95.796746,6.702696,1.7,3.280112,1.5108,1.2951,0.2157,3.0584,0.6203,-0.2573,3.3362,0.4938,-0.1559


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrd1,-440.620459,0.148369,-440.463017,0.041819,0.041685,-440.504702,298.15,-0.03713,0.14427,0.05357,0.1814,0.00791,151.526,159.943,4.5298,1231.496,321.262839,464.407217,0.902727,-0.63176,0.12919,1.3722,85.094912,95.774148,8.89756,1.700371,4.404027,1.5642,1.3606,0.2036,2.9743,0.5491,-0.2896,3.4047,0.5367,-0.0333


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrmd1,-456.671952,0.136929,-456.526108,0.041726,0.04148,-456.567588,298.15,-0.04437,0.13188,0.043755,0.17625,0.00543,144.948,152.631,5.4324,1138.958,315.992408,453.847616,0.903818,-0.61311,0.27507,1.36554,85.049716,94.647469,8.874571,1.7,4.353035,1.5684,1.3983,0.1702,3.0333,0.5547,-0.2657,3.1798,0.712,-0.0943


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrz1,-456.671574,0.136787,-456.52588,0.041605,0.041422,-456.567303,298.15,-0.04597,0.13534,0.044685,0.18131,0.00551,146.285,159.869,3.2453,1458.058,315.872386,453.680224,0.903939,-0.62054,0.08555,1.37108,85.107825,95.832257,8.905194,1.701024,4.229301,1.5697,1.3704,0.1993,2.9987,0.5534,-0.2614,3.3937,0.5073,-0.0755


Combine summary properties for all prefixes: 


,Compound_Name,E_spc (Hartree),ZPE(Hartree),H_spc(Hartree),T*S,T*qh_S,qh_G(T)_spc(Hartree),T,HOMO_Boltz,LUMO_Boltz,μ_Boltz,η_Boltz,ω_Boltz,polar_iso(Debye)_Boltz,polar_aniso(Debye)_Boltz,dipole(Debye)_Boltz,volume(Bohr_radius³/mol)_Boltz,SASA_surface_area(Å²)_Boltz,SASA_volume(Å³)_Boltz,SASA_sphericity_Boltz,NBO_charge_C1_Boltz,NBO_charge_C2_Boltz,distance_C1_C2(Å)_Boltz,%Vbur_C1_2.0Å_Boltz,%Vbur_C2_2.0Å_Boltz,Sterimol_L_C1_C2(Å)_morfeus_Boltz,Sterimol_B1_C1_C2(Å)_morfeus_Boltz,Sterimol_B5_C1_C2(Å)_morfeus_Boltz,C1_C2_bond_order_total_Boltz,C1_C2_bond_order_covalent_Boltz,C1_C2_bond_order_ionic_Boltz,C1_Co-Valency_Boltz,C1_Electro-Valency_Boltz,C1_FormalCharge_Boltz,C2_Co-Valency_Boltz,C2_Electro-Valency_Boltz,C2_FormalCharge_Boltz
0,pyrdz1,-302.982492,0.088762,-302.887089,0.035899,0.035845,-302.922934,298.15,-0.02285,0.17703,0.077090,0.19988,0.01487,87.3665,74.3950,4.3600,973.223,247.241329,331.668176,0.937200,-0.68988,0.09093,1.37265,85.098140,95.796746,6.702696,1.700000,3.280112,1.5108,1.2951,0.2157,3.0584,0.6203,-0.2573,3.3362,0.4938,-0.1559
1,pyrdz2,-302.995475,0.089531,-302.899448,0.035548,0.035546,-302.934994,298.15,-0.03489,0.19082,0.077965,0.22571,0.01347,85.7330,70.1348,2.1685,896.162,246.201207,330.946590,0.939794,-0.66830,-0.07411,1.37234,85.211131,96.816890,6.216797,1.700527,3.275876,1.5335,1.3160,0.2175,3.0303,0.5608,-0.3223,3.5204,0.3562,-0.0593
2,pyrdz3,-342.289476,0.116657,-342.164510,0.040186,0.039862,-342.204372,298.15,-0.01941,0.17521,0.077900,0.19462,0.01559,101.8870,87.9460,3.1358,1106.008,277.171300,380.768558,0.916593,-0.43009,0.09398,1.37634,92.171358,95.732180,6.710076,1.832918,3.283179,1.4930,1.3277,0.1653,3.3513,0.4731,-0.1757,3.3759,0.4330,-0.1479
3,pyrd1,-440.620459,0.148369,-440.463017,0.041819,0.041685,-440.504702,298.15,-0.03713,0.14427,0.053570,0.18140,0.00791,151.5260,159.9430,4.5298,1231.496,321.262839,464.407217,0.902727,-0.63176,0.12919,1.37220,85.094912,95.774148,8.897560,1.700371,4.404027,1.5642,1.3606,0.2036,2.9743,0.5491,-0.2896,3.4047,0.5367,-0.0333
4,pyrd2,-440.613490,0.148143,-440.456235,0.041841,0.041724,-440.497959,298.15,-0.02826,0.13999,0.055865,0.16825,0.00927,157.3340,177.7960,3.2742,1416.892,320.432780,464.008075,0.904546,-0.65663,-0.08946,1.37697,85.224044,96.800749,9.043758,1.700775,4.214565,1.5454,1.3262,0.2193,3.0693,0.6075,-0.2527,3.5178,0.3552,-0.0673


# 5. Export the data

In [12]:
# Define the filename for the Excel file
with pd.ExcelWriter(output_folder + os.sep + "Properties_postprocessed_all_prefixes.xlsx", engine="xlsxwriter") as writer:
    for prefix, properties_df_master in properties_df_master_for_all_prefixes.items():
        print(f"Writing to Excel file for prefix: {prefix}")
        all_df_master = all_df_master_for_all_prefixes[prefix]

        all_df_master.to_excel(writer, sheet_name="All_Conformer_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in all_df_master.columns:
            column_width = max(all_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = all_df_master.columns.get_loc(column)
            writer.sheets["All_Conformer_Properties_" + prefix].set_column(col_idx, col_idx, column_width)
        properties_df_master.to_excel(writer, sheet_name="Summary_Properties_" + prefix, index=False)
        # automatically adjusts the width of the columns
        for column in properties_df_master.columns:
            column_width = max(properties_df_master[column].astype(str).map(len).max(), len(column))
            col_idx = properties_df_master.columns.get_loc(column)
            writer.sheets["Summary_Properties_" + prefix].set_column(col_idx, col_idx, column_width)

Writing to Excel file for prefix: pyrdz
Writing to Excel file for prefix: pyrd
Writing to Excel file for prefix: pyrmd
Writing to Excel file for prefix: pyrz


In [13]:
# write the combined properties_df_master_for_all_prefixes to an Excel file
with pd.ExcelWriter(output_folder + os.sep + "Summary_Properties_all.xlsx", engine="xlsxwriter") as writer:
    properties_df_master_for_all_prefixes_merged.to_excel(writer, sheet_name="Summary_Properties", index=False)
    # automatically adjusts the width of the columns
    for column in properties_df_master_for_all_prefixes_merged.columns:
        column_width = max(properties_df_master_for_all_prefixes_merged[column].astype(str).map(len).max(), len(column))
        col_idx = properties_df_master_for_all_prefixes_merged.columns.get_loc(column)
        writer.sheets["Summary_Properties"].set_column(col_idx, col_idx, column_width)